In [2]:
%cd ..

/environment/Projects/dl


In [3]:
! python -m pip install deps/light-torch/

Processing ./deps/light-torch
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for light-torch: filename=light_torch-0.0.1-py3-none-any.whl size=5152 sha256=a9ea0e34a2f3103961d8fda100c5301bd14b2360037cf929e5eff6d373dbf998
  Stored in directory: /home/stefan-cristianhantig/.cache/pip/wheels/76/4b/a5/ce53c36ade717c024783dce5309d843ab5a592d06a292dafc2
Successfully built light-torch
  Attempting uninstall: light-torch
    Found existing installation: light-torch 0.0.1
    Uninstalling light-torch-0.0.1:
      Successfully uninstalled light-torch-0.0.1


In [4]:
from src.object_detection.yolov1 import YoloV1Backbone, YoloV1

In [5]:
from src.classification.models.resnet import Resnet

In [6]:
from torchsummary import summary

In [7]:
import json

In [8]:
ANNOTATION_FILE = "resources/object_detection/annotations/instances_train2014.json"

In [9]:
with open(ANNOTATION_FILE, "r") as json_file:
    coco = json.load(json_file)

In [10]:
import cv2

In [11]:
from pathlib import Path

In [12]:
import matplotlib.pyplot as plt

In [13]:
IMAGES_PATH = Path("./resources/object_detection/train2014/")

In [14]:
import numpy as np

In [15]:
from torch.utils.data.dataset import Dataset

In [16]:
from functools import lru_cache

In [17]:
from torchvision.transforms import Compose

In [18]:
from IPython.core.debugger import set_trace

In [19]:
from src.object_detection.data.coco import CocoDetectionDataset, make_id2category_map

In [20]:
from src.object_detection.utils.visualization import draw_bbox_xyhw, draw_rect_xywh_, draw_grid_

In [21]:
import torch

In [22]:
from src.object_detection.module import Module

In [23]:
from src.transform import to_rgb, to_torch, resize

In [24]:
def convert_to_torch(x):
    if not isinstance(x, np.ndarray):
        x = np.array(x)
    return torch.from_numpy(x)

In [25]:
def target_to_torch(target):
    if isinstance(target, dict):
        return {k: convert_to_torch(t) for k, t in target.items()}
    return convert_to_torch(target)

In [26]:
img_transform = Compose([
    to_torch,
])

In [27]:
coco_ds = CocoDetectionDataset(
    coco, 
    "resources/object_detection/train2014/",
    transform=to_torch,
    target_transform=target_to_torch,
    
)

In [28]:
num_classes = len(coco_ds.coco_dict["categories"])

In [29]:
#resnet = Resnet()
yolo = YoloV1(num_classes)

In [30]:
def transpose(xss):
    return list(zip(*xss))

In [31]:
from torch.nn import functional as F

In [32]:
from collections import defaultdict

In [33]:
def pad_collate(batch):
    imgs, targets = transpose(batch)
    _, h_max, w_max = np.max([np.array(img.shape) for img in imgs], axis=0)
    padded_imgs = []
    
    padded_targets = defaultdict(list)
    for img, target in zip(imgs, targets):
        target = target_to_torch(target)
        _, h, w = img.shape
        pad_h, pad_w = h_max - h, w_max - w
        img = F.pad(img, (0, pad_w, 0, pad_h), "constant", 0)
        padded_imgs.append(img)
        if len(target):
            target["boxes"] = target["boxes"] * torch.from_numpy(np.array([h / h_max, w/w_max, h/h_max, w/w_max]))
        for name, v in target.items():
            padded_targets[name].append(v)
        
    
    imgs = torch.stack(padded_imgs)
    return imgs, padded_targets
    

In [34]:
from torch.utils.data.dataloader import DataLoader

In [35]:
from src.object_detection.yolov1 import YoloV1Loss

In [36]:
yolov1_loss = YoloV1Loss()

In [37]:
from collections import defaultdict

In [38]:
def transpos_dict(dict_list):
    list_dict = defaultdict(list)
    for d in dict_list:
        for k, v in d.items():
            list_dict[k].append(v)
    return list_dict

In [39]:
def pad_images_by_hw(img, max_h, max_w):
    c, h, w = img.shape
    pad_h = max_h - h
    pad_w = max_w - w
    return F.pad(img, (0, pad_w, 0, pad_h), "constant", 0)

In [40]:
from src.object_detection.yolov1 import YoloV1Loss

In [41]:
yv1_loss = YoloV1Loss()

In [42]:
from light_torch.train import Trainer

In [43]:
from src.object_detection.module import ObjectDetectionModule

In [44]:
obj_detection = ObjectDetectionModule(yolo, yolov1_loss, coco_ds.id2name_map)

In [45]:
optimizer = torch.optim.SGD(obj_detection.parameters(), lr=0.001)

In [54]:
trainer = Trainer(
    obj_detection, 
    epochs=10,
    accumulation=4,
    optimizer=optimizer,
    name="yolov1",
)

In [47]:
subs = torch.utils.data.Subset(coco_ds, list(range(16)))

In [48]:
ds_dl = DataLoader(subs, collate_fn=pad_collate, batch_size=2, shuffle=True)

In [49]:
def fn():
    set_trace()

In [55]:
trainer.fit(ds_dl)

Train epoch  001:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [56]:
trainer.restore_checkpoint("./")

In [ ]:
trainer.fit(ds_dl)

Train epoch  006:   0%|          | 0/8 [00:00<?, ?it/s]

In [58]:
fn()

> /tmp/ipykernel_1156/341456562.py(3)fn()
      1 def fn():
      2     set_trace()
----> 3     trainer.fit(ds_dl)



ipdb>  s


--Call--
> /home/stefan-cristianhantig/.python_envs/default/lib/python3.7/site-packages/light_torch/train.py(74)fit()
     72         self.save_dir = checkpoint["save_dir"]
     73 
---> 74     def fit(
     75         self,
     76         data: DataLoader,



ipdb>  n


> /home/stefan-cristianhantig/.python_envs/default/lib/python3.7/site-packages/light_torch/train.py(81)fit()
     79         val_data: DataLoader = None,
     80     ):
---> 81         optimizer = default_if_none(optimizer, self.optimizer)
     82         epochs = default_if_none(optimizer, self.epochs)
     83         epochs = default_if_none(epochs, self.start_at_epoch + 1)



ipdb>  


> /home/stefan-cristianhantig/.python_envs/default/lib/python3.7/site-packages/light_torch/train.py(82)fit()
     80     ):
     81         optimizer = default_if_none(optimizer, self.optimizer)
---> 82         epochs = default_if_none(optimizer, self.epochs)
     83         epochs = default_if_none(epochs, self.start_at_epoch + 1)
     84         with self.module.on_fit():



ipdb>  q
